In [6]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime
import random
import os

import warnings
warnings.filterwarnings(action='ignore') 

In [7]:
# ID 제외 데이터 갖고오기 

train = pd.read_csv('../../DATA/train.csv').drop(columns=['ID'])
test  = pd.read_csv('../../DATA/test.csv').drop(columns=['ID'])
trade = pd.read_csv('../../DATA/international_trade.csv')

In [8]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])




In [9]:

train['id'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')
train

,timestamp,item,corp,loc,supply,price,id
0,2019-01-01,TG,A,J,0.0,0.0,TG A J
1,2019-01-02,TG,A,J,0.0,0.0,TG A J
2,2019-01-03,TG,A,J,60601.0,1728.0,TG A J
3,2019-01-04,TG,A,J,25000.0,1408.0,TG A J
4,2019-01-05,TG,A,J,32352.0,1250.0,TG A J
...,...,...,...,...,...,...,...
59392,2023-02-27,RD,F,J,452440.0,468.0,RD F J
59393,2023-02-28,RD,F,J,421980.0,531.0,RD F J
59394,2023-03-01,RD,F,J,382980.0,574.0,RD F J
59395,2023-03-02,RD,F,J,477220.0,523.0,RD F J


In [37]:
import pandas as pd
import numpy as np

# 연속적으로 0인 기간을 찾는 함수
def find_zero_periods(series, threshold=30):
    # 가격이 0인지 아닌지를 나타내는 binary 시퀀스 생성
    binary = (series == 0).astype(int)
    # 연속적인 구간에 대해 같은 값을 부여
    groups = binary.diff().ne(0).cumsum()
    # 각 구간의 크기 계산
    freq = groups.mul(binary).value_counts()
    # 크기가 threshold 이상인 구간 찾기
    zero_periods = freq[freq >= threshold].index
    # 해당 구간의 시작과 끝 날짜 찾기
    zero_periods_dates = [groups[groups == period].index[[0, -1]] for period in zero_periods if len(groups[groups == period].index) > 0]

    return zero_periods_dates

# 아이템별로 그룹화하고, 각 그룹에 위의 함수를 적용
def find_zero_periods_for_each_item(df, item_column='id', price_column='price', threshold=50):
    return df.groupby(item_column)[price_column].apply(find_zero_periods, threshold)


# 함수 적용
result = find_zero_periods_for_each_item(train)
result[0][1].values 
print((result[result.index== 'CB D J']))

                
        

id
CB D J    [[30153, 30392], [29066, 29295], [29411, 29624...
Name: price, dtype: object


In [43]:
index_array = []
for i in range(len(result)) :
    if i ==0 :    
        print(result[i][0][0])
    

43514
